In [25]:
import time
import json

import vk
import networkx as nx
import matplotlib.pyplot as plt
import scipy as sp

In [3]:
with open("author.txt") as f:
    login = f.readline().rstrip()
    passwd = f.readline().rstrip()

session = vk.AuthSession(app_id='6070120', user_login=login, user_password=passwd)

api = vk.API(session)

In [7]:
try:
    cache_dict = json.load(open('graph_cache.json', 'r'))
except FileNotFoundError:
    cache_dict = {}

In [18]:
def get_friends_with_sleep(user_id):
    global requests_count
    if requests_count == 3: # avoid making too many requests
        requests_count = 0
        time.sleep(1)

    try:
        friends = api.friends.get(user_id=user_id)
        requests_count += 1
        cache_dict[user_id] = friends
    except vk.exceptions.VkAPIError as err:
        if err.code == 6: # too many requests per second
            time.sleep(2)
            friends = get_friends_with_sleep(user_id)
        else:
            raise err
    return friends

In [19]:
def construct_graph(user_id, output_graph, api, waves=3):
    output_graph.add_node(user_id)
    if waves != 0:
        global cache_dict
        
        if user_id in cache_dict:
            friends = cache_dict[user_id]
        else:
            try:
                friends = get_friends_with_sleep(user_id)
                cache_dict[user_id] = friends
            except vk.exceptions.VkAPIError as err:
                if err.code == 18: # deleted or banned user
                    output_graph.remove_node(user_id)
                    friends = []
            
        for friend in friends:
            if friend not in output_graph.nodes():
                construct_graph(friend, output_graph, api, waves - 1)
            output_graph.add_edge(user_id, friend)

In [21]:
# start with my id of 10938483 for now
requests_count = 0
main_graph = nx.DiGraph()
start_id = 10938483
construct_graph(start_id, main_graph, api)

In [22]:
len(main_graph.nodes())

498290

In [23]:
with open('graph_cache.json', 'w') as f:
    json.dump(cache_dict, f)

In [28]:
nx.adjacency_matrix(main_graph)

<498290x498290 sparse matrix of type '<class 'numpy.int64'>'
	with 564850 stored elements in Compressed Sparse Row format>